In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.quantization as tq
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
MODEL_NAME      = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
PRUNE_AMOUNT    = 0.30      # 30% magnitude pruning
BATCH_SIZE      = 5
MAX_NEW_TOKENS  = 50
PROMPT          = (
    "In a world increasingly driven by artificial intelligence, the ability to interpret "
    "large language models efficiently is crucial for both research and deployment."
)
LOGDIR_BASELINE = "./profiler_logs/baseline"


In [3]:
def load_model(device: torch.device):
    """
      - Load the tokenizer: AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
      - Load the model: AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float32)
      - Move model to `device` and call .eval()
      - Return (tokenizer, model)
    """
    # 1) Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    # 2) Load FP32 model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float32
    )

    # 3) Move to device and set to eval mode
    model.to(device)
    model.eval()

    # 4) Return both
    return tokenizer, model

In [4]:
def make_batch(tokenizer, prompt: str, device: torch.device):
    """
      - Duplicate `prompt` BATCH_SIZE times into a list of strings
      - Tokenize with padding and truncation: tokenizer(..., return_tensors="pt")
      - Move inputs to `device`
      - Return the tokenized inputs
    """
    # 1) Replicate prompt
    texts = [prompt] * BATCH_SIZE

    # 2) Tokenize with padding & truncation
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    # 3) Move to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    return inputs


In [5]:
def profile_inference(model, inputs, logdir: str, label: str):
    """
      - Create `logdir` if it doesn't exist
      - Use `torch.profiler.profile` (CPU & CUDA, record_shapes, profile_memory, with_stack)
      - Inside the profiler, wrap the call to `model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)` in `record_function(label)`
      - After profiling, print:
          1) Top-3 ops by "self_cpu_time_total"
          2) Top-3 ops by "self_cuda_time_total"
          3) Total CPU vs CUDA self-time in milliseconds
      - Note: use `prof.key_averages().table(...)` and sum over `evt.self_cpu_time_total`, `evt.self_cuda_time_total`
      - Traces should be saved automatically by `tensorboard_trace_handler`
    """
    # 1) Ensure log directory exists
    os.makedirs(logdir, exist_ok=True)

    # 2) Run profiler
    with profile(
        activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
        record_shapes=True,
        profile_memory=True,
        with_stack=True,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(logdir)
    ) as prof:
        with record_function(label):
            _ = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)

    # 3) Print top‐3 operators by CPU self time
    print(f"\n=== {label} Top-3 ops by CPU self time ===")
    print(prof.key_averages().table(
        sort_by="self_cpu_time_total", row_limit=3
    ))

    # 4) Print top‐3 operators by CUDA self time
    print(f"\n=== {label} Top-3 ops by CUDA self time ===")
    print(prof.key_averages().table(
        sort_by="self_cuda_time_total", row_limit=3
    ))

    # 5) Summarize total CPU vs CUDA self times
    events     = prof.key_averages()
    total_cpu  = sum(evt.self_cpu_time_total for evt in events)
    total_cuda = sum(getattr(evt, "self_cuda_time_total", 0) for evt in events)
    print(f"\n=== {label} Total self-time ===")
    print(f"CPU  : {total_cpu/1e3:.2f} ms")
    print(f"CUDA : {total_cuda/1e3:.2f} ms")

    print(f"\nTrace files for '{label}' written to: {logdir}")

In [6]:
def apply_pruning_and_quant(model: nn.Module):
    """
      - On CPU, apply `prune.l1_unstructured(..., amount=PRUNE_AMOUNT)` to every nn.Linear weight
      - Call `prune.remove(...)` to make masks permanent
      - Then apply `torch.quantization.quantize_dynamic` on {nn.Linear} with dtype=torch.qint8
      - Return the quantized model
    """
    # 1) Prune on CPU
    model.cpu()
    for module in model.modules():
        if isinstance(module, nn.Linear):
            # zero out PRUNE_AMOUNT fraction of the smallest‐magnitude weights
            prune.l1_unstructured(module, name="weight", amount=PRUNE_AMOUNT)
            # make the pruning permanent
            prune.remove(module, "weight")

    # 2) Dynamic 8-bit quantization
    quantized = tq.quantize_dynamic(
        model,
        {nn.Linear},
        dtype=torch.qint8
    )

    return quantized

In [7]:
def start():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1) Load & benchmark baseline
    tokenizer, model = load_model(device)
    batch_inputs     = make_batch(tokenizer, PROMPT, device)
    print("Profiling inference…")
    profile_inference(model, batch_inputs, LOGDIR_BASELINE, label="Baseline")


In [8]:
start()

Profiling inference…


STAGE:2025-09-06 02:57:50 10150:10150 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-09-06 02:57:57 10150:10150 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-09-06 02:57:57 10150:10150 ActivityProfilerController.cpp:324] Completed Stage: Post Processing



=== Baseline Top-3 ops by CPU self time ===
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               Baseline        36.95%        1.494s       100.00%        4.043s        4.043s       0.000us         0.00%        1.458s        1.458s          

# Exercise 1: Measure Model Size & Nonzeros

## Concept: After pruning and quantization, the student should measure how the model structure has changed.

In [15]:
import os, shutil, torch
import numpy as np

def _safe_state_dict(model):
    """
    TODO:
      - Get the model state_dict
      - Build a new dict that keeps ONLY valid tensor-like entries
      - If value is a torch.Tensor -> detach, move to CPU
      - If value is a numpy.ndarray -> convert to torch tensor
      - Drop any other types (dtype objects, None, etc.)
      - Return the filtered dict
    """
    raise NotImplementedError("Implement _safe_state_dict")


def _dir_size_mb(path: str) -> float:
    """
    TODO:
      - Walk the directory tree starting at path
      - Sum file sizes (os.path.getsize for each file)
      - Return total size in megabytes (divide by 1024**2)
    """
    raise NotImplementedError("Implement _dir_size_mb")


def measure_model_sparsity_and_size(model, name: str = "model"):
    """
    TODO:
      - Initialize counters total=0, nonzero=0
      - Iterate over model.modules()
          * If module has a weight tensor, accumulate total params and nonzero count
      - Print sparsity stats: nonzero/total and % sparse
      - Create/reset a temporary directory for saving
      - Try: if model has save_pretrained, call it
      - Except: fall back to _safe_state_dict(model) + torch.save(...)
      - Compute dir size with _dir_size_mb
      - Print final disk size
    """
    raise NotImplementedError("Implement measure_model_sparsity_and_size")



# Exercise 2 — Batch size & prompt-length sensitivity (latency/throughput)

## Concept: Show how decoding cost scales with batch size and max_new_tokens on your baseline vs optimized models.

In [19]:
def _model_device(model: torch.nn.Module) -> torch.device:
    """
    TODO:
      - Inspect model parameters and buffers.
      - If any parameter exists, return its device.
      - Else if any buffer exists, return its device.
      - Else default to torch.device("cpu").
    """
    raise NotImplementedError("Implement _model_device")


def sweep_batch_and_length(tokenizer, model_dict, _unused_device,
                           prompt, batch_sizes=(1, 2, 4),
                           gen_lengths=(16, 64, 128),
                           runs=3):
    """
    TODO: Implement a device-safe sweep to benchmark models.

    Steps:
      1. Tokenize the prompt ONCE on CPU for a single sample.
      2. For each model in model_dict:
          a. Detect its device using _model_device.
          b. For each batch size:
              - Tile the single sample to match batch size.
              - Move tensors to the model's device.
          c. For each generation length:
              - Run multiple trials (runs).
              - Use torch.cuda.synchronize() before/after if device is CUDA.
              - Time model.generate with max_new_tokens=L, use_cache=True.
              - Compute latency and throughput (tokens/sec across the whole batch).
      3. Append results as dicts with keys:
          ["model", "batch_size", "max_new_tokens", "latency_s", "tokens_per_sec", "device"].
      4. Return results as a pandas DataFrame.
    """
    raise NotImplementedError("Implement sweep_batch_and_length")


# Exercise 3 — Operator hot-spots before vs after optimization

## Concept: Use torch.profiler to summarize top operators for baseline vs optimized. Students see where time is spent (e.g., attention matmuls vs layernorm) and how it shifts after pruning/quant.

In [ ]:
# --- Robust operator hot-spot summary (event-based, device-safe) ---
from torch.profiler import profile, ProfilerActivity
import torch


def _model_device(model: torch.nn.Module) -> torch.device:
    """
    Return the device of the model parameters/buffers.
    Students: Implement device extraction logic.
    """
    raise NotImplementedError("Return the device where the model is located")


def _to_device_batch(batch: dict, device: torch.device) -> dict:
    """
    Move all tensors in batch to the specified device.
    Students: Implement tensor device transfer logic.
    """
    raise NotImplementedError("Move all tensors in the batch to the given device")


def top_ops_summary(model, batch_inputs, device_unused, with_cuda=True, top_k=10, label="run"):
    """
    Profile the model for a single short run and summarize top CPU/CUDA operators.

    Returns:
        {
            "label": str,
            "device": str,
            "cpu": [(op_name, time_ms), ...],
            "cuda": [(op_name, time_ms), ...],
            "counts": dict
        }
    """
    # TODO: Step 1 – Get model device
    mdev = _model_device(model)

    # TODO: Step 2 – Move batch to device
    enc = _to_device_batch(batch_inputs, mdev)

    # TODO: Step 3 – Select profiler activities
    raise NotImplementedError("Setup profiler activities (CPU always, CUDA if available)")

    # TODO: Step 4 – Run model under profiler
    # Hint: use torch.inference_mode(), torch.cuda.synchronize(), and model.generate()
    raise NotImplementedError("Profile the model execution with torch.profiler")

    # TODO: Step 5 – Extract event statistics
    # Hint: prof.key_averages(), self_cpu_time_total, self_cuda_time_total
    raise NotImplementedError("Summarize operator self-times into top-k CPU/CUDA ops")

    # TODO: Step 6 – Return structured dictionary
    raise NotImplementedError("Return dict with label, device, cpu, cuda, counts")


In [25]:
# === Run All Exercises ===
def run_all_exercises():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1) Load baseline model & tokenizer
    tokenizer, baseline_model = load_model(device)
    batch_inputs = make_batch(tokenizer, PROMPT, device)

    # 2) Create optimized model (prune + quantize)
    optimized_model = apply_pruning_and_quant(baseline_model)

    print("\n================ Exercise 1: Sparsity & Size ================")
    measure_model_sparsity_and_size(baseline_model, "baseline")
    measure_model_sparsity_and_size(optimized_model, "optimized")

    print("\n================ Exercise 2: Batch Size & Length Sweep ================")
    df_sweep = sweep_batch_and_length(
        tokenizer,
        {"baseline": baseline_model, "optimized": optimized_model},
        device,
        PROMPT,
        batch_sizes=(1, 2),
        gen_lengths=(16, 64),
        runs=2
    )
    display(df_sweep)

    print("\n================ Exercise 3: Operator Hot-Spots ================")
    base_ops = top_ops_summary(baseline_model, batch_inputs, device, with_cuda=True, top_k=8, label="baseline")
    opt_ops  = top_ops_summary(optimized_model, batch_inputs, device, with_cuda=True, top_k=8, label="optimized")
    
    print("Baseline device:", base_ops["device"], "| events:", base_ops["counts"])
    print("Optimized device:", opt_ops["device"], "| events:", opt_ops["counts"])
    print("\nTop CPU ops (baseline):", base_ops["cpu"])
    print("Top CPU ops (optimized):", opt_ops["cpu"])
    
# Call it
run_all_exercises()



================ Exercise 1: Sparsity & Size ================
[baseline] Nonzero params: 789722330/1100048384 (28.21% sparse)
[baseline] Disk size: 4196.37 MB
[optimized] Nonzero params: 65628160/65628160 (0.00% sparse)
[optimized] Disk size: 250.45 MB

================ Exercise 2: Batch Size & Length Sweep ================


,model,batch_size,max_new_tokens,latency_s,tokens_per_sec,device
0,baseline,1,16,1.564530,10.228149,cpu
1,baseline,1,64,5.054521,12.667585,cpu
2,baseline,2,16,1.840176,17.395002,cpu
3,baseline,2,64,6.051661,21.179314,cpu
4,optimized,1,16,0.679012,23.564206,cpu
5,optimized,1,64,2.692659,23.768656,cpu
6,optimized,2,16,0.767073,41.722865,cpu
7,optimized,2,64,2.942899,43.494537,cpu



================ Exercise 3: Operator Hot-Spots ================


STAGE:2025-09-06 03:25:30 10150:10150 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-09-06 03:25:35 10150:10150 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-09-06 03:25:35 10150:10150 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-09-06 03:25:45 10150:10150 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-09-06 03:25:47 10150:10150 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-09-06 03:25:47 10150:10150 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


Baseline device: cpu | events: {'evt_total': 74, 'cpu_nonzero': 8, 'cuda_nonzero': 0}
Optimized device: cpu | events: {'evt_total': 72, 'cpu_nonzero': 8, 'cuda_nonzero': 0}

Top CPU ops (baseline): [('aten::mm', 3373.148), ('aten::mul', 69.753), ('aten::cat', 56.771), ('aten::_scaled_dot_product_flash_attention_for_cpu', 52.607), ('aten::copy_', 49.95), ('aten::add', 37.566), ('aten::matmul', 37.352), ('aten::silu', 30.704)]
Top CPU ops (optimized): [('quantized::linear_dynamic', 601.199), ('aten::cat', 49.253), ('aten::mul', 45.053), ('aten::_scaled_dot_product_flash_attention_for_cpu', 45.007), ('aten::copy_', 34.219), ('aten::add', 25.002), ('aten::empty', 21.191), ('aten::silu', 20.13)]
